In [45]:
import os.path as op
import sys
sys.path.insert(0, op.expanduser("~/projects/io3d"))

import numpy as np
import io3d
import pandas as pd
import matplotlib.pyplot as plt



In [46]:
patient_id_path = io3d.datasets.join_path("patient_id.csv")
input_path =  io3d.datasets.join_path("chk")
# input_path =  io3d.datasets.join_path("chk/2018-04-15/**")
output_path =  io3d.datasets.join_path("anon/")

In [47]:
df = pd.read_csv(patient_id_path)
# df

# Prepare humanized hash

In [48]:
import hashlib

m = hashlib.sha256()
m.update("as".encode())
# m.digest_size = 8
# str(m.digest())
display(m.hexdigest())
m.update("ii".encode())
display(m.hexdigest())
m.update("as".encode())
display(m.hexdigest())

'f4bf9f7fcbedaba0392f108c59d8f4a38b3838efb64877380171b54475c2ade8'

'518c63c8b9995a21737699708801b3e2f8ebe7701fe6b6ab6dfc9e118f4ec5fa'

'4dc24ef5b819958c98ba325ae1eb7548a3a6bf5079442da14d1efee23af4964a'

In [49]:
m.update(bytes(str(-5).encode()))

In [50]:
import humanhash
hh = '03'
humanhash.humanize(hh, words=2)

'alaska'

In [51]:
def get_hexd_and_human_digest(data, words_number=4):
    if type(data) != str:
        data = str(data)
        
    inbytes = data.encode()
    m = hashlib.sha256()
    m.update(inbytes)
    hexdigest = m.hexdigest()
    
    humandigest = humanhash.humanize(hexdigest, words=words_number) 
    return hexdigest, humandigest
    

In [52]:
str == type ("as")

True

In [53]:
# sha256
# 2 words: max_test_number = 600
# 3 words: max_test_number = 7100
# 4 words: max_test_number = 76000
max_test_number = 76000
words_number = 4
numbers = list(range(0, max_test_number))
hashs = []
for id_int in numbers:
    hexdigest, humandigest = get_hexd_and_human_digest(id_int, words_number=words_number)

    hashs.append(humandigest)
# display(hashs)
len_orig = len(numbers)
len_hdigest = len(np.unique(hashs))
assert(len_orig == len_hdigest)

In [54]:
hashs

['hot-missouri-glucose-september',
 'wyoming-william-louisiana-kilo',
 'fifteen-river-sad-michigan',
 'ack-double-oregon-nevada',
 'river-may-indigo-friend',
 'mike-nitrogen-purple-black',
 'twenty-failed-spaghetti-berlin',
 'finch-fanta-zulu-lamp',
 'lemon-beer-mockingbird-oxygen',
 'earth-cola-six-nuts',
 'carbon-neptune-washington-georgia',
 'lion-carolina-six-juliet',
 'early-comet-fourteen-asparagus',
 'artist-monkey-april-zulu',
 'coffee-one-wisconsin-william',
 'hawaii-glucose-triple-muppet',
 'football-sixteen-one-jupiter',
 'winner-fifteen-mike-twenty',
 'coffee-river-lake-kilo',
 'missouri-glucose-jersey-social',
 'charlie-triple-colorado-wolfram',
 'autumn-august-gee-iowa',
 'zulu-oregon-mike-cat',
 'pennsylvania-ack-florida-uranus',
 'rugby-uniform-cold-cold',
 'magnesium-west-bravo-arkansas',
 'edward-echo-quiet-october',
 'butter-social-muppet-emma',
 'cup-seventeen-network-massachusetts',
 'speaker-item-robin-delaware',
 'victor-glucose-arkansas-south',
 'mobile-crazy-fa

# Walk directory search for studies

In [55]:
import glob
display(input_path)
dirlist = glob.glob(op.join(input_path, "2018/**/"))
# dirlist = glob.glob(input_path + "/")
# dirlist

'E:data/medical/orig\\chk'

In [56]:
dirpath, trail = op.split(dirlist[0])
dirpath2, dirname = op.split(dirpath)

In [57]:
# df["Study ID"][:10]

In [58]:
def get_id(datadir, df):
    dirpath, trail = op.split(datadir)
    dirpath2, dirname = op.split(dirpath)
    found = df["Study ID"].astype(str) == dirname
    study_id = dirname
    
    if any(found):
#         display(df[found][["Simple Patient ID", "Study ID"]])
        patient_id = df[found][["Simple Patient ID"]].iat[0,0]
        return study_id, patient_id
        
    else:
        return study_id, None
    

In [59]:
for datadir in dirlist:
    study_id, patient_id = get_id(datadir, df)
        

In [60]:
def create_path(path_base, patient_id, study_id, series_id, ext=".raw"):
    patient_hex, patient_human = get_hexd_and_human_digest(patient_id, words_number=4)
    study_hex, study_human = get_hexd_and_human_digest(study_id, words_number=2)
    fn = patient_human + "_" + str(study_human) + "_" + str(series_id) + "_" + study_hex + "_v0" + ext
    fn_full = op.join(path_base, fn)
    return fn_full

# Walk study directory

In [61]:
def export_study(datadir, outputdir, df, dry_read=False, dry_write=False, ext=".mhd", return_prefiltered=True, 
                 filter_no_patient_id=True, filter_min_count=None, filter_max_count=None
                ):
    
    study_id, patient_id = get_id(datadir, df)
    
    if filter_no_patient_id and patient_id is None:
        if return_prefiltered:
            return None, None
        else:
            return None
    
    dd = io3d.dcmreaddata.DicomDirectory(datadir)
    series_stats = dd.get_stats_of_series_in_dir()
    series_list = series_stats.keys()
    print(datadir, series_list)
    # get pandas df
    study_df = pd.DataFrame(series_stats).transpose()
    # filtration
    if return_prefiltered:
        study_df_all = study_df.copy()
        
    if filter_min_count is not None:
        study_df = study_df[study_df.Count > filter_min_count]
    if filter_max_count is not None:
        study_df = study_df[study_df.Count < filter_max_count]
    
    display(series_list)
    for i in range(len(study_df)):
        series_number = study_df.iloc[i]["SeriesNumber"]
        # for series_number in series_list:
        dr = io3d.DataReader()
        if dry_read:
            data3d = io3d.datasets.generate_face([50, 50, 50]).astype(np.int16) * 150
            metadata = {"voxelsize_mm": [1.0, 0.5, 0.5]}
            datap = {
                "data3d": data3d,
                "metadata": metadata
            }
        else:
            datap = dr.Get3DData(datapath=datadir, series_number=series_number)

        fn_full = create_path(path_base=outputdir, patient_id=patient_id, series_id=series_number, study_id=study_id, ext=ext)
        print(patient_id, study_id, series_number, datadir, fn_full)
        if not dry_write:
            io3d.write(path=fn_full, data3d=datap)
        
    if return_prefiltered:
        return study_df, study_df_all
    else:
        return study_df

# Export from dir list

In [62]:
# %debug
# def export_with_hash(datadir, output_path)
dry_read = True
dry_write = True
# dry_read = False
# dry_write = False
df
dfdir_filtered = []
dfdir_all = []
for datadir in dirlist:
#     print(datadir)
    try:
        dff, dfa = export_study(
            datadir, output_path, df, 
            filter_max_count=200, filter_min_count=10,
            return_prefiltered=True, dry_read=dry_read, dry_write=dry_write
        )
        if dff is not None:
            dfdir_filtered.append(dff)
            dfdir_all.append(dfa)
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(e)
#     display(dfdir_all)

SyntaxError: invalid syntax (<ipython-input-62-a03ee06d375a>, line 2)

In [ ]:
pd.concat(dfdir_all, ignore_index=True)[["Count", "Modality", "SeriesNumber", "voxelsize_mm"]]


In [ ]:

pd.concat(dfdir_filtered, ignore_index=True)[["Count", "Modality", "SeriesNumber", "voxelsize_mm"]]
# dd = io3d.dcmreaddata.DicomDirectory(datadir)
# series_stats = dd.get_stats_of_series_in_dir()

In [ ]:
# dirlist